<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5c42af6718ca6137d18d5261a848a9ecc0973214ee008dc7a1054227de7a4b91
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-01 14:42:12
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.40 C
-------------------
Today PnL: 1.44 L (1.03%)
Current PnL: -21.39 L (-14.2%)
CY Booked + Current PnL: -9.95 L (-6.61%)
-------------------
Total profit:  1.26 L
Total loss:  -22.65 L
-------------------
Total Booked + Current PnL: 17.03 L (13.77%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.16%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.70 L (63.27%)
Deployed:  1.24 C
Current:  1.40 C
CAGR/XIRR %: 7.66%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.54,113531.0,13027.0,16076.0,0.16,12.96,14.16,28.96,79.0,0.81,0.81,51.33,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.60,37.0,M-SC,2.45,83447.0,-17330.0,17424.0,0.32,-17.20,20.88,0.09,245.0,-0.99,0.60,9.63,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,7.95,43.0,H-MC,3.38,186460.0,21790.0,21536.0,-1.64,13.23,11.55,26.31,89.0,1.01,1.34,22.27,X40N,NTT,AC
42,ITC,452.00,-38.77,47.0,H-LC,2.03,198192.0,-1946.0,22832.0,0.93,-0.97,11.52,10.44,4.0,-0.09,1.42,4.54,X40,NTT,FMCG
57,RELIANCE,1533.00,-14.79,41.0,H-LC,3.70,213408.0,3042.0,25737.0,0.29,1.45,12.06,13.68,37.0,0.12,1.53,18.21,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,68.86,77.0,H-SC,1.74,155295.0,-23776.0,89512.0,14.57,-13.28,57.64,36.71,135.0,-0.27,1.11,54.08,XR,NTT,BANKS
26,GREENPANEL,537.00,232.69,58.0,M-SC,1.37,148645.0,-34433.0,113416.0,10.36,-18.81,76.30,43.14,230.0,-0.30,1.06,39.82,XY24,NTT,MISC
64,SHALBY,327.00,1234.93,68.0,M-SC,14.23,181462.0,319.0,45148.0,5.96,0.18,24.88,25.10,235.0,0.01,1.30,45.42,XY24,NTT,HEALTHCARE
73,TATAMOTORS,1065.00,-17.55,60.0,M-LC,2.29,260761.0,-15762.0,125843.0,5.61,-5.70,48.26,39.81,54.0,-0.13,1.87,24.97,XY24,NTT,AUTO
71,TANLA,1963.11,-21.15,52.0,H-SC,17.94,211216.0,-59545.0,389503.0,5.10,-21.99,184.41,121.86,133.0,-0.15,1.51,65.23,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-47.86,23.0,M-SC,1.20,88125.0,-3374.0,82124.0,-4.67,-3.69,93.19,86.07,223.0,-0.04,0.63,27.97,XR,NTT,DURABLES
55,RAJOOENG,143.10,-45.66,20.0,H-SC,14.26,82440.0,-20060.0,60659.0,-3.60,-19.57,73.58,39.61,114.0,-0.33,0.59,0.00,AR,ATH,MISC
3,ALKYLAMINE,4546.37,-12.07,29.0,H-SC,10.69,81842.0,-19121.0,113654.0,-3.40,-18.94,138.87,93.63,148.0,-0.17,0.59,21.90,SR,ATH,CHEMICALS
81,VAIBHAVGBL,521.00,47.11,43.0,H-SC,5.80,127701.0,-55074.0,166139.0,-2.61,-30.13,130.10,60.77,125.0,-0.33,0.91,14.38,XR,NTT,JEWELLERY
38,INDIGOPNTS,1408.00,102.57,30.0,M-SC,1.51,132680.0,-41879.0,41914.0,-1.89,-23.99,31.59,0.02,221.0,-1.00,0.95,15.41,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1234.93,68.0,M-SC,14.23,181462.0,319.0,45148.0,5.96,0.18,24.88,25.10,235.0,0.01,1.30,45.42,XY24,NTT,HEALTHCARE
35,ICICIPRULI,790.0,-21.80,44.0,H-MC,2.21,134617.0,-1171.0,43926.0,1.73,-0.86,32.63,31.48,107.0,-0.03,0.96,11.15,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.25,43.0,M-LC,5.74,150350.0,-595.0,110056.0,1.31,-0.39,73.20,72.51,53.0,-0.01,1.08,5.60,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,65.0,H-SC,11.46,129924.0,-12555.0,32078.0,3.82,-8.81,24.69,13.70,163.0,-0.39,0.93,53.35,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,56.0,H-SC,1.58,224145.0,-45522.0,81858.0,1.00,-16.88,36.52,13.47,138.0,-0.56,1.61,13.37,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.60,37.0,M-SC,2.45,83447.0,-17330.0,17424.0,0.32,-17.20,20.88,0.09,245.0,-0.99,0.60,9.63,OX40N,NTT,DURABLES
66,SIS,528.0,2101.55,44.0,H-SC,4.04,88855.0,-22177.0,45787.0,0.94,-19.97,51.53,21.26,156.0,-0.48,0.64,20.05,OX40N,NTT,MISC
18,CERA,9475.0,-22.91,35.0,H-SC,1.62,140346.0,-35557.0,77583.0,-0.92,-20.21,55.28,23.89,149.0,-0.46,1.01,21.21,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1234.93,68.0,M-SC,14.23,181462.0,319.0,45148.0,5.96,0.18,24.88,25.10,235.0,0.01,1.30,45.42,XY24,NTT,HEALTHCARE
60,SAIL,228.00,47.55,60.0,M-MC,11.53,233142.0,8180.0,159469.0,0.68,3.64,68.40,74.53,192.0,0.05,1.67,37.06,XY24,BTT,STEEL
32,HINDZINC,730.22,28.52,64.0,M-LC,3.86,209366.0,4290.0,108284.0,-0.24,2.09,51.72,54.89,52.0,0.04,1.50,26.69,X5K,ATH,METALS
14,BLUESTARCO,2080.00,7.95,43.0,H-MC,3.38,186460.0,21790.0,21536.0,-1.64,13.23,11.55,26.31,89.0,1.01,1.34,22.27,X40N,NTT,AC
84,VOLTAS,1530.00,-3.32,42.0,H-MC,2.84,202800.0,11058.0,26709.0,0.88,5.77,13.17,19.69,99.0,0.41,1.45,12.96,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.54,113531.0,13027.0,16076.0,0.16,12.96,14.16,28.96,79.0,0.81,0.81,51.33,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,9.45,191604.0,11796.0,100611.0,0.95,6.56,52.51,62.52,88.0,0.12,1.37,42.88,XR,NTT,BANKS
32,HINDZINC,730.22,28.52,64.0,M-LC,3.86,209366.0,4290.0,108284.0,-0.24,2.09,51.72,54.89,52.0,0.04,1.50,26.69,X5K,ATH,METALS
37,INDIAMART,4850.92,-54.64,30.0,H-SC,10.01,126447.0,3111.0,130645.0,0.80,2.52,103.32,108.45,119.0,0.02,0.91,25.44,AR,ATH,MISC
86,WIPRO,420.00,-14.25,43.0,M-LC,5.74,150350.0,-595.0,110056.0,1.31,-0.39,73.20,72.51,53.0,-0.01,1.08,5.60,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.55,28.0,H-LC,5.83,205691.0,-46077.0,99287.0,-0.54,-18.30,48.27,21.13,27.0,-0.46,1.47,10.64,X40,ATH,PAINTS
83,VBL,671.64,-21.55,32.0,H-LC,9.26,282443.0,-33399.0,146051.0,-0.23,-10.57,51.71,35.67,5.0,-0.23,2.02,1.81,X40N,ATH,FMCG
74,TCS,4476.75,-30.93,34.0,H-LC,12.38,274781.0,-71175.0,146046.0,1.20,-20.57,53.15,21.64,1.0,-0.49,1.97,1.20,X40,ATH,IT
30,HCLTECH,1943.91,-4.09,36.0,H-LC,10.23,218780.0,-23144.0,86418.0,0.61,-9.57,39.50,26.15,8.0,-0.27,1.57,3.39,X40,ATH,IT
44,JIOFIN,387.00,-5.12,37.0,H-LC,12.89,257785.0,-9580.0,75041.0,2.23,-3.58,29.11,24.48,48.0,-0.13,1.85,49.42,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-6.94,44.0,H-LC,1.63,230985.0,-15166.0,94380.0,-0.48,-6.16,40.86,32.18,16.0,-0.16,1.65,27.66,X200,ATH,IT
65,SIEMENS,4671.5,-3.04,43.0,H-LC,1.98,155815.0,-30280.0,77767.0,-0.41,-16.27,49.91,25.51,15.0,-0.39,1.12,14.82,AR,ATH,ELECTRICAL
42,ITC,452.0,-38.77,47.0,H-LC,2.03,198192.0,-1946.0,22832.0,0.93,-0.97,11.52,10.44,4.0,-0.09,1.42,4.54,X40,NTT,FMCG
57,RELIANCE,1533.0,-14.79,41.0,H-LC,3.70,213408.0,3042.0,25737.0,0.29,1.45,12.06,13.68,37.0,0.12,1.53,18.21,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.28,46.0,H-LC,3.81,273125.0,7699.0,49081.0,1.24,2.90,17.97,21.40,11.0,0.16,1.96,10.44,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,34.0,H-LC,12.38,274781.0,-71175.0,146046.0,1.20,-20.57,53.15,21.64,1.0,-0.49,1.97,1.20,X40,ATH,IT
83,VBL,671.64,-21.55,32.0,H-LC,9.26,282443.0,-33399.0,146051.0,-0.23,-10.57,51.71,35.67,5.0,-0.23,2.02,1.81,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.09,36.0,H-LC,10.23,218780.0,-23144.0,86418.0,0.61,-9.57,39.50,26.15,8.0,-0.27,1.57,3.39,X40,ATH,IT
42,ITC,452.00,-38.77,47.0,H-LC,2.03,198192.0,-1946.0,22832.0,0.93,-0.97,11.52,10.44,4.0,-0.09,1.42,4.54,X40,NTT,FMCG
40,INFY,2275.00,-20.67,40.0,H-LC,8.19,308488.0,-4400.0,176085.0,0.45,-1.41,57.08,54.87,3.0,-0.02,2.21,5.15,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.04,43.0,H-LC,1.98,155815.0,-30280.0,77767.0,-0.41,-16.27,49.91,25.51,15.0,-0.39,1.12,14.82,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,47.0,H-LC,2.03,198192.0,-1946.0,22832.0,0.93,-0.97,11.52,10.44,4.0,-0.09,1.42,4.54,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.55,28.0,H-LC,5.83,205691.0,-46077.0,99287.0,-0.54,-18.30,48.27,21.13,27.0,-0.46,1.47,10.64,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.18,43.0,H-LC,3.97,207391.0,2891.0,38533.0,1.73,1.41,18.58,20.25,10.0,0.08,1.49,10.27,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-14.79,41.0,H-LC,3.70,213408.0,3042.0,25737.0,0.29,1.45,12.06,13.68,37.0,0.12,1.53,18.21,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.93,34.0,H-LC,12.38,274781.0,-71175.0,146046.0,1.20,-20.57,53.15,21.64,1.0,-0.49,1.97,1.20,X40,ATH,IT
40,INFY,2275.00,-20.67,40.0,H-LC,8.19,308488.0,-4400.0,176085.0,0.45,-1.41,57.08,54.87,3.0,-0.02,2.21,5.15,X40,BTT,IT
42,ITC,452.00,-38.77,47.0,H-LC,2.03,198192.0,-1946.0,22832.0,0.93,-0.97,11.52,10.44,4.0,-0.09,1.42,4.54,X40,NTT,FMCG
83,VBL,671.64,-21.55,32.0,H-LC,9.26,282443.0,-33399.0,146051.0,-0.23,-10.57,51.71,35.67,5.0,-0.23,2.02,1.81,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,45.0,H-LC,7.69,249240.0,-12379.0,131599.0,0.37,-4.73,52.80,45.57,7.0,-0.09,1.79,6.05,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,46.0,L-SC,40.44,80832.0,-32717.0,72765.0,4.13,-28.81,90.02,35.27,268.0,-0.45,0.58,97.84,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,35.0,H-SC,12.29,90000.0,-7980.0,29583.0,-0.88,-8.14,32.87,22.05,152.0,-0.27,0.64,30.82,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.54,113531.0,13027.0,16076.0,0.16,12.96,14.16,28.96,79.0,0.81,0.81,51.33,MH,ATH,METALS
78,UJJIVANSFB,60.00,105.37,65.0,H-SC,11.46,129924.0,-12555.0,32078.0,3.82,-8.81,24.69,13.70,163.0,-0.39,0.93,53.35,OX40N,NTT,BANKS
26,GREENPANEL,537.00,232.69,58.0,M-SC,1.37,148645.0,-34433.0,113416.0,10.36,-18.81,76.30,43.14,230.0,-0.30,1.06,39.82,XY24,NTT,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-187.50,81.0,M-SC,8.35,151425.0,2205.0,141976.0,4.59,1.48,93.76,96.62,208.0,0.02,1.08,63.87,XY25,NTT,FINANCE
64,SHALBY,327.00,1234.93,68.0,M-SC,14.23,181462.0,319.0,45148.0,5.96,0.18,24.88,25.10,235.0,0.01,1.30,45.42,XY24,NTT,HEALTHCARE
69,SURYODAY,240.00,68.86,77.0,H-SC,1.74,155295.0,-23776.0,89512.0,14.57,-13.28,57.64,36.71,135.0,-0.27,1.11,54.08,XR,NTT,BANKS
60,SAIL,228.00,47.55,60.0,M-MC,11.53,233142.0,8180.0,159469.0,0.68,3.64,68.40,74.53,192.0,0.05,1.67,37.06,XY24,BTT,STEEL
50,NATIONALUM,244.55,-42.83,63.0,H-MC,12.54,113531.0,13027.0,16076.0,0.16,12.96,14.16,28.96,79.0,0.81,0.81,51.33,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.06
1,25,43.16
2,50,74.88


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.21
LC    32.13
MC    23.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.96
X40      14.49
X40N     11.21
XR       11.07
XY25     10.46
AR        8.11
OX40N     7.79
X200      1.65
MH        1.61
X5K       1.50
SR        1.17
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.96
H-LC    25.52
H-MC    20.40
M-SC    13.80
M-LC     5.58
M-MC     2.98
L-SC     1.45
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.83,-6.25,41.32
IT,12.52,-20.28,85.86
FINANCE,10.37,-16.36,69.22
MISC,6.98,-18.02,83.63
BANKS,6.41,-11.40,70.00
PAINTS,5.55,-19.11,36.87
ELECTRICAL,5.34,-10.56,49.36
INSURANCE,3.75,-5.58,45.05
AUTO,3.57,-15.37,70.52


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3253550.0,22
XR,1363338.0,14
AR,1200768.0,9
X40,825328.0,10
X40N,640268.0,9
OX40N,560904.0,10
XY25,476131.0,7
SR,259100.0,2
X5K,108284.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3750670.0,29
M-SC,1505378.0,17
H-LC,1259325.0,15
H-MC,1256249.0,15
M-LC,371704.0,4
M-MC,367297.0,2
L-SC,259759.0,3
L-MC,60293.0,1
L-LC,39876.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1235038.0      6
M-SC       XY24       846931.0      7
H-SC       AR         821781.0      5
           XR         790416.0      7
H-MC       XY24       575879.0      4
H-LC       X40        530668.0      5
M-MC       XY24       367297.0      2
H-SC       X40N       315285.0      4
           SR         259100.0      2
           OX40N      256626.0      4
H-LC       X40N       236123.0      3
H-MC       X40        219466.0      4
H-LC       AR         209366.0      2
H-MC       XY25       180532.0      2
L-SC       XR         171056.0      2
M-SC       AR         169621.0      2
M-LC       XY24       153364.0      2
M-SC       XY25       141976.0      1
           OX40N      140750.0      4
           XR         130906.0      2
H-LC       XY25       113747.0      3
M-LC       XR         110056.0      1
           X5K        108284.0      1
H-MC       XR         100611.0      1
H-LC       X200        94380.0      1
H-MC       X40N        88860.0      2
L-SC       OX40N       88703.0      1
M-SC       X40         75194.0      1
H-LC       XY24        75041.0      1
H-MC       OX40N       74825.0      1
H-SC       MH          72424.0      1
L-MC       XR          60293.0      1
L-LC       XY25        39876.0      1
H-MC       MH          16076.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
